In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import transforms, models
import pytorch_lightning as pl
from pytorch_lightning.utilities.model_summary import ModelSummary
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import wandb
import torchmetrics

In [2]:
wandb.login()
wandb.init(
      mode='disabled',
      # Set the project where this run will be logged
      project="histopathologic-cancer-classifier", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name="Test4", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0002,
      "data_size": 0.3,
      "batch_size":32,
      })

wandb: Currently logged in as: respwill. Use `wandb login --relogin` to force relogin


In [3]:
class DataModule(pl.LightningDataModule):
    def __init__(self, batch_size, train, val):
        super().__init__()
        self.batch_size = batch_size
        self.dims = (3, 96, 96)
        self.num_classes = 1
        self.train = train
        self.val = val
    
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)       

In [4]:
class ImageEncoder(pl.LightningModule):
    def __init__(self, input_shape, num_classes, learning_rate=2e-4):
        super().__init__()
        
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=20, out_channels=30, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=30, out_channels=40, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.ReLU()
        )
        n_sizes = self._get_conv_output((3,96,96))
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_sizes, 1500),
            nn.ReLU(),
            nn.Linear(1500, 500),
            nn.ReLU(),
            nn.Linear(500, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )
        
        self.accuracy = torchmetrics.Accuracy(task='binary')
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.dense(x)
        return torch.sigmoid(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y)
        pred = torch.argmax(y_hat, dim=1)
        return {"loss":loss, 'pred':pred, 'y':y}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        pred = torch.cat([x['pred'] for x in outputs]).to(device=torch.device('cpu'))
        y = torch.cat([x['y'] for x in outputs]).to(torch.int32)
        y = torch.squeeze(y)
        avg_acc = self.accuracy(pred, y)
        self.log("train_loss", avg_loss, prog_bar=True)
        self.log("train_acc", avg_acc, prog_bar=True)
        wandb.log({'train_loss': avg_loss, 'learning_rate': self.lr_schedulers().get_last_lr()[0], 'train_acc':avg_acc})
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        pred = torch.argmax(y_hat, dim=1)
        val_loss = F.binary_cross_entropy(y_hat, y)
        return {'val_loss':val_loss,'pred':pred, 'y':y}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#         print([x['pred'] for x in outputs])
        pred = torch.cat([x['pred'] for x in outputs]).to(device=torch.device('cpu'))
#         print([x['y'] for x in outputs])
        y = torch.cat([x['y'] for x in outputs]).to(torch.int32)
        y = torch.squeeze(y)
#         print('pred:', pred, pred.shape)
#         print('y:', y, y.shape)
        avg_acc = self.accuracy(pred, y)
        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_acc", avg_acc, prog_bar=True)
        wandb.log({'val_loss': avg_loss, 'val_acc':avg_acc})
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.00001)
        return [optimizer], [scheduler]
    
    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self.encoder(input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        n_size = n_size
        return n_size
    


In [5]:
class CancerDataset(Dataset):
    def __init__(self, label_dir, img_dir):
        self.img_labels = pd.read_csv(label_dir)
        self.img_dir = img_dir
        self.transform = transforms.ToTensor()
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        # label file has two columns: id, label
        # to read each images from dir
        img_path = self.img_dir + '/'+ self.img_labels.iloc[idx,0] + '.tif'
        # read image as numpy array and normalize it
        image = Image.open(img_path)
        image_array = self.transform(image) / 255.0
        
        # read label
        label = self.img_labels.iloc[idx, 1].astype('float32')
        label = torch.Tensor([label])
        # return image array and label
        return image_array, label

In [6]:
data = CancerDataset(label_dir='/Users/haeinpark/Downloads/histopathologic-cancer-detection/train_labels.csv',
                          img_dir='/Users/haeinpark/Downloads/histopathologic-cancer-detection/train/')

In [7]:
train_n = int(len(data) * wandb.config['data_size'] * 0.7)
val_n = int(len(data) * wandb.config['data_size'] * 0.3)

In [8]:
train_n, val_n

(46205, 19802)

In [9]:
def train_val_dataset(dataset, train_split=0.75, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), train_size=train_split, test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [10]:
dataset = train_val_dataset(data, train_split=train_n, val_split=val_n)

In [11]:
# train_dataloader = DataLoader(dataset['train'], batch_size=wandb.config['batch_size'])
# val_dataloader = DataLoader(dataset['val'], batch_size=wandb.config['batch_size'])

In [12]:
dm = DataModule(batch_size=wandb.config['batch_size'], train=dataset['train'], val=dataset['val'])

In [13]:
im_encoder = ImageEncoder(input_shape=dm.dims, num_classes=dm.num_classes, learning_rate=wandb.config['learning_rate'])

In [14]:
ModelSummary(im_encoder, max_depth=-1)

   | Name       | Type           | Params
-----------------------------------------------
0  | encoder    | Sequential     | 50.9 K
1  | encoder.0  | Conv2d         | 760   
2  | encoder.1  | ReLU           | 0     
3  | encoder.2  | Conv2d         | 5.0 K 
4  | encoder.3  | ReLU           | 0     
5  | encoder.4  | MaxPool2d      | 0     
6  | encoder.5  | ReLU           | 0     
7  | encoder.6  | Conv2d         | 15.0 K
8  | encoder.7  | ReLU           | 0     
9  | encoder.8  | Conv2d         | 30.0 K
10 | encoder.9  | ReLU           | 0     
11 | encoder.10 | MaxPool2d      | 0     
12 | encoder.11 | ReLU           | 0     
13 | dense      | Sequential     | 18.1 M
14 | dense.0    | Flatten        | 0     
15 | dense.1    | Linear         | 17.3 M
16 | dense.2    | ReLU           | 0     
17 | dense.3    | Linear         | 750 K 
18 | dense.4    | ReLU           | 0     
19 | dense.5    | Linear         | 25.1 K
20 | dense.6    | ReLU           | 0     
21 | dense.7    | Linear    

In [15]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./check_point/', filename='{epoch}-{train_loss:.4f}-{val_loss:.4f}', 
    monitor="val_loss", mode="min", save_top_k=5
)

In [16]:
trainer = pl.Trainer(accelerator='cpu',
                     devices=1,
                    max_epochs=200,
                    callbacks=[checkpoint_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/haeinpark/miniconda3/envs/boulder/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [17]:
trainer.fit(im_encoder, dm)

/Users/haeinpark/miniconda3/envs/boulder/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /Users/haeinpark/Documents/GitHub/histopathologic-cancer-detection/check_point exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name     | Type           | Params
--------------------------------------------
0 | encoder  | Sequential     | 50.9 K
1 | dense    | Sequential     | 18.1 M
2 | accuracy | BinaryAccuracy | 0     
--------------------------------------------
18.2 M    Trainable params
0         Non-trainable params
18.2 M    Total params
72.672    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/haeinpark/miniconda3/envs/boulder/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/haeinpark/miniconda3/envs/boulder/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/haeinpark/miniconda3/envs/boulder/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
